In [2]:

from appyter import magic
magic.init(lambda _ = globals: _())

# GeneSet Library Set Appyter
This appyter is designed to perform basic statistics, analysis, and visualizations on a Gene Matrix Transpose (.GMT) file. This will allow bioinformatics researchers to analyze relationships between many different gene sets from several gene set libraries.
 To create your own GMT file, please see Enrichr. Enrichr, hosted by the Ma'ayan Laboratory at Mt. Sinai Icahn School of Medicine, is a collection of geneset libraries. 

In [3]:
import numpy as np 
import pandas as pd
import itertools 
import bokeh
import pathlib
from bokeh.plotting import figure, show, save, output_file
from maayanlab_bioinformatics.enrichment import crisp
# from Umap_Visualization import UMAP_Visualization
import os


install tqdm for progress bar


In [4]:
%%appyter hide_code


{% do SectionField(name='GMTSubmission', title='1. Submit a GMT file', subtitle = 'Sumbit a GMT (Gene Matrix Transpose file) for analysis.', img = 'bulb.png') %}
{% do SectionField (name = 'UMAP visualization', title = '2. Scatterplot Visualization', subtitle= 'Visualize relative Geneset similarities on an interactive scatterplot', img = 'bulb.png') %}
{% do SectionField(name = 'Pairwise Similarity Table', title = '3. GeneSet Pairwise Similarity Table', subtitle = '##TODO : FINISH Explore How Similar a Pair of Gene Sets are to One Another. Use the search bar to search for Terms that are of interest. Use the sorting arrows to see', img = 'bulb.png') %}
{% do SectionField(name = 'Pairwise Similarity Search', title = '4. GeneSet Pair Search', subtitle = '###TODO', img = 'bulb.png')%}
{% do SectionField(name = 'GMT Descriptive Statistics', title = '5. Descriptive Statistics', subtitle = '#TODO', img = 'bulb.png') %}


In [12]:
%%appyter code_exec

{% set gs = TabField(
    name='gs_type',
    label='Gene Sets',
    default='Upload',
    choices={'Upload': [
            MultiFileField(
                name='gs',
                label='Gene Set Files',
                default=['Geneshot_PainGenes.gmt'],
                examples={
                    'example.gmt': url_for('static', filename = 'Geneshot_PainGenes.gmt')
                }
            ),
        ],},
    
section = 'GMTSubmission',)%}

gs = {{gs.value[0]}}


```python

gs = ['Geneshot_PainGenes.gmt']
```

In [17]:
def series_to_list(gene_list):
    ##helper function to convert a gene sereis to a gene list
    ret_list = []
    for genes in gene_list:
        if type(genes) is str:
            ret_list.append(genes)
        else: ##pd series case
            genes = genes.tolist()        
            ret_list.append(' '.join(genes))
    return ret_list

In [20]:
%%appyter code_exec

def load_sets(*files):
    ''' Load a set of files into pairs of labeled sets
    '''
    lst = []
    for file in map(pathlib.Path, files):
        path = pathlib.Path(file)
        with open(path) as f:
            lines = f.readlines()
            for line in lines:
                parsed_line = line.split('\t')
                term, library, genes = parsed_line[0], parsed_line[1], parsed_line[2:]
                genes[-1] = genes[-1][:-2] ##trim off newline regex '\n'
                lst.append((term,  library, ' '.join(genes)))
        zip_lst = [list(i) for i in zip(*lst)]
        term, library, genes = zip_lst[0], zip_lst[1], zip_lst[2]
        genes = series_to_list(genes)
        
        df = pd.DataFrame({'Genes': genes, 'Library': library}, index = term)
        return df
        
                
            
#         if file.suffix == '.gmt':
#             for line in map(str.strip, file.open('r')):
                
                

```python
def load_sets(*files):
    ''' Load a set of files into pairs of labeled sets
    '''
    lst = []
    for file in map(pathlib.Path, files):
        path = pathlib.Path(file)
        with open(path) as f:
            lines = f.readlines()
            for line in lines:
                parsed_line = line.split('\t')
                term, library, genes = parsed_line[0], parsed_line[1], parsed_line[2:]
                genes[-1] = genes[-1][:-2] ##trim off newline regex '\n'
                lst.append((term,  library, ' '.join(genes)))
        zip_lst = [list(i) for i in zip(*lst)]
        term, library, genes = zip_lst[0], zip_lst[1], zip_lst[2]
        genes = series_to_list(genes)
        df = pd.DataFrame({'Genes': genes, 'Library': library}, index = term)
        return df
#         if file.suffix == '.gmt':
#             for line in map(str.strip, file.open('r')):
```

In [24]:
load_sets(*gs)['Genes']

A_Geneshot_PainGenes_GeneRIF_AssociatedGenes                        TRPV1 OPRM1 TRPA1 COMT SCN9A TNF IL6 IL1B CRP ...
B_Geneshot_PainGenes_AutoRIF_AssociatedGenes                        TNF TRPV1 CRP FOS PTGS2 NGF TRPA1 BDNF CD34 PO...
C_Geneshot_PainGenes_GeneRIF_PredictedGenes_AutoRIF-CoOccurrence    OPRD1 TRPV1 TRPA1 SCN9A OPRM1 TRPM8 TACR1 OPRK...
Name: Genes, dtype: object